In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.14088821411132812
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 62.18985523603938
200000 104.0043098167393
300000 105.4328037682911
400000 103.97439146215667
500000 101.38043669229299
600000 101.98698896487574
700000 106.87294443331467
800000 101.77851006647711
900000 101.39826146637469
1000000 100.38336779712087
1100000 101.17648894214955
1200000 100.54785495619308
1300000 82.25239924938225
1400000 40.3153161530232
1500000 99.74004352216652
1600000 99.5598051086423
1700000 99.91909549738727
1800000 98.80950579455892
1900000 100.18754205504821
2000000 104.74513708813276
2100000 100.6685120054564
2200000 99.29326883531762
2300000 100.11290928995848
2400000 100.5743929558261
2500000 102.38882906725233
2600000 96.58011908567818
2700000 24.35422850876304
2800000 95.82654515913495
2900000 96.15545479191776
3000000 95.7279

29400000 106.13166686658198
29500000 106.73244864200107
29600000 102.53962426107599
29700000 105.40083465854553
29800000 104.57698886502305
29900000 105.01994920779879
30000000 102.81651009562495
30100000 101.48168311272306
30200000 90.18490818911741
30300000 37.060331169522314
30400000 97.40371298826395
30500000 102.1477424061218
30600000 101.63208348928151
30700000 106.15916026326667
30800000 100.31163982528193
30900000 100.5924807307069
31000000 104.94865764741425
31100000 108.18599846480684
31200000 103.79405907265604
31300000 98.86744470264387
31400000 50.41298111183458
31500000 109.56062995419714
31600000 111.03947381075277
31700000 110.56531019777296
31800000 50.10999438939314
31900000 94.00206894266542
32000000 109.30909312956477
32100000 108.76718313765329
32200000 109.65634140235426
32300000 105.49064261696205
32400000 103.3204478903572
32500000 106.44595055271087
32600000 104.04323350709579
32700000 104.05254499680012
32800000 101.14461468418017
32900000 24.79630497962717
33

59200000 23.221775472366513
59300000 99.0998861814464
59400000 98.08511127332892
59500000 96.68290404126512
59600000 99.05696436087278
59700000 101.81460282793321
59800000 99.17133515048909
59900000 103.0530018324835
60000000 98.96481760248263
60100000 43.3715810650976
60200000 105.4420370464649
60300000 105.07331967256727
60400000 105.90666389771445
60500000 103.1907134511971
60600000 103.27425694725923
60700000 102.51828087872553
60800000 102.25483954215785
60900000 65.24074428292703
61000000 78.24103180876448
61100000 100.34314819431874
61200000 97.08051591402094
61300000 96.91051991650882
61400000 103.46523935412749
61500000 94.17408856573492
61600000 93.11177414169758
61700000 91.31175070791733
61800000 33.15373573268097
61900000 86.0247758566655
62000000 91.82495218372144
62100000 92.03488263589328
62200000 96.61515400475253
62300000 93.91855790983416
62400000 95.6419961010944
62500000 98.48111971024326
62600000 102.40686296214895
62700000 34.41036241340043
62800000 107.189724411

89300000 106.00328086038974
89400000 108.98637195931059
89500000 107.98279539583035
89600000 104.86431092220353
89700000 105.649095388603
89800000 106.01116135974208
89900000 101.51811696978345
90000000 77.44875185176629
90100000 56.54725697233637
90200000 94.58130644260034
90300000 92.61183341595795
90400000 93.3340900839478
90500000 94.57290462010768
90600000 93.39244810020676
90700000 40.40128643694857
90800000 80.00814699649776
90900000 92.1694457627853
91000000 91.09071914499381
91100000 91.60706020113506
91200000 92.79394979970388
91300000 90.0425562152927
91400000 33.34181682420898
91500000 100.28625860240238
91600000 96.46835963520479
91700000 101.47449302914063
91800000 99.84445236336707
91900000 101.88868538513353
92000000 107.5381501745598
92100000 100.12629071752437
92200000 98.12764511308626
92300000 98.05997361047748
92400000 96.71708901685749
92500000 96.90050112202756
92600000 95.94103312114181
92700000 30.562370750594383
92800000 87.90677988136784
92900000 88.891735725

119000000 85.94157460903969
119100000 78.87861955677978
119200000 38.8330604549996
119300000 84.09964040469961
119400000 88.53938511850036
119500000 85.51570764114544
119600000 45.298680722588564
119700000 69.10971979049224
119800000 85.20296817310314
119900000 85.30665615544729
120000000 87.3546339874684
120100000 33.94821788148664
120200000 88.32254107159977
120300000 87.16485323974028
120400000 86.8612100120024
120500000 91.17435041254541
120600000 91.54548319536542
120700000 95.4104452445971
120800000 92.6483219157392
120900000 96.62819016425581
121000000 94.15184867922414
121100000 96.83948351276851
121200000 97.70815438186807
121300000 97.16215870042855
121400000 89.96597907967671
121500000 90.2129975149821
121600000 89.50817603487405
121700000 87.25987642490001
121800000 42.82934134104516
121900000 72.81693129102291
122000000 86.392930091226
122100000 86.13082709667869
122200000 65.48977156793984
122300000 45.98859231384531
122400000 88.11755635998081
122500000 86.80085868770247

...

983000000 3.75989793091892
983100000 4.888709863218737
983200000 7.9290349299311575
983300000 7.534434507013851
983400000 7.5032059237257895
983500000 6.2378327353614065
983600000 5.556552593244475
983700000 1.8197281475051106
983800000 6.593389177240023
983900000 6.155653735740794
984000000 6.5288363130804115
984100000 6.598774124272668
984200000 5.730865031347127
984300000 3.4669317547271405
984400000 2.8866369578211386
984500000 6.6237247636693155
984600000 5.734544765039964
984700000 6.3955566452118955
984800000 6.623083908608027
984900000 5.36558542018952
985000000 2.8019622062869427
985100000 4.467155304223939
985200000 6.555160416726417
985300000 6.365994289927053
985400000 6.118160214453284
985500000 6.635327723014477
985600000 4.160487318896526
985700000 2.5612967445655443
985800000 5.636767271588315
985900000 6.501873365231985
986000000 6.535161901412031
986100000 5.674382845134738
986200000 5.725065060663678
986300000 3.0485209868354612
986400000 4.256352589897718
986500000 

1011400000 2.790999618991244
1011500000 2.6852698590593516
1011600000 3.2244230489723344
1011700000 2.8740161170264007
1011800000 2.8756969870643614
1011900000 1.4601518326857263
1012000000 1.6922364242469845
1012100000 2.805794310685286
1012200000 2.8459006706345518
1012300000 3.044226792061301
1012400000 2.4845699996252346
1012500000 2.244344662433798
1012600000 0.8594423092110436
1012700000 2.0794874059580355
1012800000 2.5490333230790836
1012900000 2.7300977218686464
1013000000 2.5047197833992825
1013100000 2.3870201854213913
1013200000 1.465267613861823
1013300000 1.0753780979933845
1013400000 2.4049826343302905
1013500000 2.317296312222757
1013600000 2.7028585473161297
1013700000 2.2171138003190847
1013800000 2.1740770754084884
1013900000 0.8374027548310777
1014000000 1.6537799204650587
1014100000 2.1519613651532032
1014200000 2.4445329408749235
1014300000 2.3952033628946303
1014400000 2.1527013380566915
1014500000 1.5777787351124148
1014600000 0.886682114327105
1014700000 2.2238

1037300000 3.5949102404133564
1037400000 3.599801570616744
1037500000 2.815398829858267
1037600000 3.5393195563554696
1037700000 3.419040221143454
1037800000 2.714793011022822
1037900000 3.7527386848032114
1038000000 3.1723108054275553
1038100000 2.650148903941089
1038200000 3.5708025058782678
1038300000 3.1700036949405206
1038400000 2.772692027452038
1038500000 3.654730286344286
1038600000 3.265462774494281
1038700000 2.767528844155436
1038800000 3.64920435101315
1038900000 3.379835546933664
1039000000 2.9406474491951404
1039100000 3.3903642879708316
1039200000 3.637918675701509
1039300000 2.9227843018397803
1039400000 2.9187949981701222
1039500000 3.828875166349767
1039600000 3.3390428117218653
1039700000 2.73869303028085
1039800000 3.459349125284135
1039900000 3.5265447684411226
1040000000 3.2797500421326986
1040100000 3.757935803676551
1040200000 4.8697627336274465
best so far: 0
type: [1, 3, 1, 2, 23] 138
cases of this type: 6716184
1040300000 2.654489876166587
1040400000 1.677093

1058500000 2.463072925683705
1058600000 1.809011129106756
1058700000 1.6390422328958507
1058800000 2.0224017939420107
1058900000 2.0510810748749595
1059000000 1.6505916908622862
1059100000 1.7987545412214734
1059200000 2.420765359494982
1059300000 1.4159303887987076
1059400000 1.654500585555971
1059500000 2.224052858853658
1059600000 1.7569498341697871
1059700000 1.441830268330705
1059800000 2.1665892963479676
1059900000 1.8457124966267369
1060000000 1.6676770400212624
1060100000 1.875751169847868
1060200000 2.230500039568399
1060300000 1.3383888165011009
1060400000 1.7889146673194056
1060500000 2.1889598454802948
1060600000 1.6748187296559216
1060700000 1.4514873884407824
1060800000 2.084740949255714
1060900000 1.7555041484641531
1061000000 1.3192359368475675
1061100000 1.9765245635615845
1061200000 1.9944536013834158
1061300000 1.3101454676897686
1061400000 1.679652932112528
1061500000 2.0393588225498913
1061600000 1.3711849875055624
1061700000 1.633151881341602
1061800000 1.97907410

1085000000 0.3694307045560261
1085100000 0.33441741248164375
1085200000 0.3332218047985236
1085300000 0.32505160974366404
1085400000 0.3365456640555044
1085500000 0.332882588346519
1085600000 0.3365854044674098
1085700000 0.34811073413335414
1085800000 0.19694069238077402
1085900000 0.10288589638347624
1086000000 0.10213644207535519
1086100000 0.1011593535142978
1086200000 0.10048110584548846
1086300000 0.13485713798110022
1086400000 0.37941895429593525
1086500000 0.31699151705761747
1086600000 0.3027468125537932
1086700000 0.2998337570700407
1086800000 0.2848639072949476
1086900000 0.2877672805252168
1087000000 0.28119021128947536
1087100000 0.2847622662126091
1087200000 0.3140987885670159
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.26581131735600827
best so far: 0
type: [2, 1, 23, 1, 3] 138
cases of this type: 42849
1087400000 0.0800673527165234
best so far: 0
type: [2, 1, 23, 3, 1] 138
cases of this type: 14283
best so far: 0
type: [2, 1, 69, 1,